In [1]:
import torch
import time
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
class ScienceRAG:
    def __init__(self, 
                 qdrant_host: str = "localhost", 
                 qdrant_port: int = 6333,
                 collection_name: str = "nlp2025_chunks",
                 embed_model: str = "Qwen/Qwen3-Embedding-0.6B",
                 llm_model: str = "Qwen/Qwen2.5-1.5B-Instruct"):
        
        self.collection_name = collection_name
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Инициализация RAG на устройстве: {self.device.upper()}")

        # 1. Подключение к базе знаний
        self.client = QdrantClient(qdrant_host, port=qdrant_port)
        
        # 2. Загрузка ретривера (Эмбеддинги)
        print(f"Загрузка ретривера: {embed_model}...")
        self.encoder = SentenceTransformer(embed_model, trust_remote_code=True, device="cpu")
        
        # 3. Загрузка генератора (LLM)
        print(f"🧠 Загрузка LLM: {llm_model}...")
        self.tokenizer = AutoTokenizer.from_pretrained(llm_model)
        
        # ИЗМЕНЕНИЯ ЗДЕСЬ:
        # 1. Убрали device_map="auto"
        # 2. Сменили bfloat16 на float16
        # 3. Явно отправили .to(self.device)
        self.model = AutoModelForCausalLM.from_pretrained(
            llm_model,
            torch_dtype=torch.float16, 
            attn_implementation="sdpa" # Включаем быстрое внимание (Scaled Dot Product Attention)
        ).to(self.device)
        print("Система готова к работе!\n")

    
    def _retrieve(self, query: str, top_k: int = 5):
        """Внутренний метод: поиск векторов"""
        query_vector = self.encoder.encode(query, convert_to_numpy=True)
        
        search_result = self.client.query_points(
            collection_name=self.collection_name,
            query=query_vector,
            limit=top_k,
            with_payload=True
        )
        
        # Упрощаем структуру для генератора
        return [point.payload for point in search_result.points]


    def _format_context(self, chunks) -> str:
        """Собирает найденные чанки в один текст для промпта"""
        formatted_text = ""
        for i, chunk in enumerate(chunks):
            # Адаптируй ключи под свой payload JSON
            title = chunk.get('title', 'Unknown Title')
            text = chunk.get('text', chunk.get('abstract', '')) # fallback если нет text
            
            formatted_text += f"Document [{i+1}]\nTitle: {title}\nContent: {text}\n\n"
        return formatted_text


    def answer(self, query: str, top_k: int = 5) -> str:
            t0 = time.time()
            
            # --- ЭТАП 1: РЕТРИВ ---
            print(f"🔍 Ищу информацию...")
            retrieved_chunks = self._retrieve(query, top_k)
            t1 = time.time()
            print(f"⏱️  Retrieval (Encode + Search): {t1 - t0:.4f} сек")
            
            if not retrieved_chunks:
                return "Нет информации."
    
            # --- ЭТАП 2: ПОДГОТОВКА ---
            context = self._format_context(retrieved_chunks)
            system_prompt = (
                "You are a helpful scientific assistant. "
                "Use the provided context to answer the user's question. "
                "If the context doesn't contain the answer, admit it."
            )
            messages = [
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {query}"}
            ]
            
            text_input = self.tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            model_inputs = self.tokenizer([text_input], return_tensors="pt").to(self.device)
            t2 = time.time()
            
            # --- ЭТАП 3: ГЕНЕРАЦИЯ ---
            print("✍️  Генерирую ответ...")
            with torch.no_grad():
                generated_ids = self.model.generate(
                    **model_inputs,
                    max_new_tokens=512,
                    temperature=0.3,
                    top_p=0.9,
                    do_sample=True
                )
            t3 = time.time()
            print(f"⏱️  Generation (LLM): {t3 - t2:.4f} сек")
    
            response = self.tokenizer.batch_decode(
                [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)],
                skip_special_tokens=True
            )[0]
            
            print(f"⏱️  Total: {t3 - t0:.2f} сек")
            return response

In [3]:
rag = ScienceRAG()

# Запрос
user_query = "How are Graph Neural Networks used in drug discovery?"

answer = rag.answer(user_query, top_k=5)

print("\n" + "="*40)
print("OTBET:")
print(answer)
print("="*40)

Инициализация RAG на устройстве: CUDA
Загрузка ретривера: Qwen/Qwen3-Embedding-0.6B...
🧠 Загрузка LLM: Qwen/Qwen2.5-1.5B-Instruct...


`torch_dtype` is deprecated! Use `dtype` instead!


Система готова к работе!

🔍 Ищу информацию...
⏱️  Retrieval (Encode + Search): 0.2322 сек
✍️  Генерирую ответ...
⏱️  Generation (LLM): 10.5557 сек
⏱️  Total: 10.80 сек

OTBET:
Graph Neural Networks (GNNs) are used in drug discovery through various applications such as:

1. **Drug Target Binding Affinity Prediction**: The Hybrid Graph-Transformer framework introduced by Xiao et al. ([2024]) uses GNNs to integrate both graph-based and sequence-based representations, achieving superior performance compared to state-of-the-art methods on benchmark datasets.

2. **Multi-Objective Molecule Optimization**: The Latent Prompt Transformer developed by Kong et al. ([2024]) incorporates latent prompts within a unified architecture to achieve state-of-the-art performance in multi-objective molecule optimization and drug-like molecule generation.

These applications leverage the ability of GNNs to process and analyze complex structures like molecules, making them valuable tools in advancing our unde

In [4]:
# Запрос
user_query = "How are Graph Neural Networks used in social media?"

answer = rag.answer(user_query, top_k=5)

print("\n" + "="*40)
print("OTBET:")
print(answer)
print("="*40)

🔍 Ищу информацию...
⏱️  Retrieval (Encode + Search): 0.2035 сек
✍️  Генерирую ответ...
⏱️  Generation (LLM): 7.5100 сек
⏱️  Total: 7.72 сек

OTBET:
Based on the information provided in Document [4], Graph Neural Networks (GNNs) are being utilized in the field of knowledge-graph-guided language understanding, particularly within collaborative multi-agent question answering systems. Specifically, they are described as "large-scale pretrained GNN backbones" that aim to capture broadly reusable structural/semantic patterns. This suggests their application in generating synthetic graphs from text data, which can then be used to inform or enhance language model routing decisions. The use of GNNs here appears to leverage their ability to process relational data effectively, making them suitable for tasks involving complex interactions between entities represented as nodes in a graph structure.
